In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline
sns.set_style('whitegrid')
sns.set_context('paper')

In [2]:
def select(query):
    
    conn = sqlite3.connect('./data/lending-club-loan-data/database2.sqlite')
    cursor = conn.cursor()
    temp_df = pd.DataFrame(cursor.execute(query).fetchall())
    temp_df.columns = list(map(lambda x: x[0], cursor.description))
    conn.close()
    
    return temp_df.copy()

In [3]:
features_train = select('SELECT * FROM FEATURES_TRAIN')
targets_train = select('SELECT * FROM TARGETS_TRAIN').loan_status
features_test = select('SELECT * FROM FEATURES_TEST')
targets_test = select('SELECT * FROM TARGETS_TEST').loan_status

In [4]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import ADASYN

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import make_scorer
from sklearn.model_selection import GridSearchCV

In [5]:
# undersampling ratio, SMOTE oversampling ratio, ADASYN oversampling ratio. all 1:1

ratios = [{0:len(features_train[~targets_train.astype(bool)]),\
               1:len(features_train[~targets_train.astype(bool)])},\
         {0:len(features_train[targets_train.astype(bool)]),\
                1:len(features_train[targets_train.astype(bool)])},\
         {0:len(features_train[targets_train.astype(bool)]),\
                1:len(features_train[targets_train.astype(bool)])}]

In [6]:
# sklearn's Pipeline class requires that intermediary steps have fit and transform methods. our re-samplers
# do not have transforms, so we must define a custom pipeline for this specific application.
# the function will be fairly limited and specific to our needs for simplicity/effectiveness

# the function runs through the whole pipeline twice: without hyperparam optimization and with hyperparam optimization

# 'model' argument should be a tuple w/ the first entry as the model's name as a string; the second entry a class instance 
# model names should be: LR, GNB, KNN, or RF for the function to properly work..

# 8 results for each algorithm. note that resampling is w/o a deterministic seed, so results may vary on the same call

def Pipeline0(features,targets,model,resample_ratios,metric):
    
    resamplers = [None,RandomUnderSampler,SMOTE,ADASYN]
    resampler_names = ['UnderSamp','SMOTE','ADASYN']
    output = {}
    output['model'] = model[0]
    i = 0
    
    for resampler in resamplers:
        if resampler == None:
            final_features = features.copy()
            final_targets = targets.copy()
            tn,fp,fn,tp = confusion_matrix(targets_test,model[1].fit(final_features,final_targets).predict(features_test)).ravel()
            
            if metric == 'neg':
                output['neg_prec'] = tn/(tn+fn)
                output['neg_recall'] = tn/(tn+fp)
            elif metric == 'pos':
                output['pos_prec'] = tp/(tp+fp)
                output['pos_recall'] = tp/(tp+fn)
            
        else:
            final_features, final_targets = resampler(ratio=resample_ratios[i]).fit_sample(features,targets)
            tn,fp,fn,tp = confusion_matrix(targets_test,model[1].fit(final_features,final_targets).predict(features_test)).ravel()
            
            if metric == 'neg':
                output[resampler_names[i]+'_prec'] = tn/(tn+fn)
                output[resampler_names[i]+'_recall'] = tn/(tn+fp)
            elif metric == 'pos':
                output[resampler_names[i]+'_prec'] = tp/(tp+fp)
                output[resampler_names[i]+'_recall'] = tp/(tp+fn)
                
            i+=1
    
    return output

In [7]:
# optimize hyperparameters for F1 score on negative class, since that is the area we're looking to improve

def neg_f1(targets_true,targets_predicted):
    tn, fp, fn, tp = confusion_matrix(targets_true,targets_predicted).ravel()
    precision = tn/(tn+fn)
    recall = tn/(tn+fp)
    return 2*(precision*recall)/(precision+recall)

def pos_f1(targets_true,targets_predicted):
    tn, fp, fn, tp = confusion_matrix(targets_true,targets_predicted).ravel()
    precision = tp/(tp+fp)
    recall = tp/(tp+fn)
    return 2*(precision*recall)/(precision+recall)

# the metric to optimize for in the regularized runs of the pipeline will determine the appropriate score function

score_dict = {'neg':neg_f1,'pos':pos_f1}

In [8]:
# we will leave out RF. we saw in the analysis of regularization that changing the params from the defaults
# greatly reduces model performance across all metrics

# we can actually just incorporate this function into pipeline0, using control flow..

def Pipeline1(features,targets,model,resample_ratios,metric):
    
    resamplers = [None,RandomUnderSampler,SMOTE,ADASYN]
    resampler_names = ['UnderSamp','SMOTE','ADASYN']
    output = {}
    output['model'] = 're'+model[0]
    i = 0
    
    if model[0] == 'LR':
        params = {'C':[.001,.01,.1,1,10,100]}
    elif model[0] == 'GNB': 
        params = {'priors':[[0.1,0.9],[0.2,0.8],[0.3,0.7],[0.4,0.6],[0.5,0.5],[0.6,0.4]]}
    elif model[0] == 'KNN':
        params = {'n_neighbors':list(range(3,8))}
    elif model[0] == 'RF':
        params = {'min_samples_split':[2,8,32],'min_samples_leaf':[1,16,32]}
    
    for resampler in resamplers:
        if resampler == None:
            final_features = features.copy()
            final_targets = targets.copy()
            clf = GridSearchCV(model[1],param_grid=params,scoring=make_scorer(score_dict[metric]),return_train_score=True)\
                                .fit(final_features,final_targets)
            tn,fp,fn,tp = confusion_matrix(targets_test,clf.best_estimator_.predict(features_test)).ravel()
            
            if metric == 'neg':
                output['neg_prec'] = tn/(tn+fn)
                output['neg_recall'] = tn/(tn+fp)
            elif metric == 'pos':
                output['pos_prec'] = tp/(tp+fp)
                output['pos_recall'] = tp/(tp+fn)
            
        else:
            final_features, final_targets = resampler(ratio=resample_ratios[i]).fit_sample(features,targets)
            clf = GridSearchCV(model[1],param_grid=params,scoring=make_scorer(score_dict[metric]),return_train_score=True)\
                                .fit(final_features,final_targets)
            tn,fp,fn,tp = confusion_matrix(targets_test,clf.best_estimator_.predict(features_test)).ravel()
            
            if metric == 'neg':
                output[resampler_names[i]+'_prec'] = tn/(tn+fn)
                output[resampler_names[i]+'_recall'] = tn/(tn+fp)
            elif metric == 'pos':
                output[resampler_names[i]+'_prec'] = tp/(tp+fp)
                output[resampler_names[i]+'_recall'] = tp/(tp+fn)
                
            i+=1
    
    return output

In [9]:
# defines a pipeline process at a higher level of abstraction, which runs both the unregularized and regularized 
# version of the algorithm, returning the final desired 2x8 matrix with results across different sampling methods.

# the column w/ the metric name as the column name is the un-resampled version

def PipelineLayer1(model,metric,ratio_dicts):
    unregularized = Pipeline0(features=features_train,targets=targets_train,\
                             model=model,resample_ratios=ratio_dicts,metric=metric)
    
    regularized = Pipeline1(features=features_train,targets=targets_train,\
                            model=model,resample_ratios=ratio_dicts,metric=metric)
    return pd.DataFrame(unregularized,index=[0]).set_index('model')\
                    .append(pd.DataFrame(regularized,index=[0]).set_index('model')).copy()

In [10]:
# algs_list should be a list of tuples (tuples formatted as the 'model' argument are entered into the lower level pipelines)
# e.g. ('LR',LogisticRegression())

# 'metric' is changed to pos/neg. we always want to study precision and recall together. for the actual analysis,
# we would run this function twice, to view study the performance of the models on negative classes and positive classes
# we avoid returning all the metrics at once, because we're left with a table of 16 columns..too much information to
# easily digest from a human standpoint

def PipelineLayer2(model_list,metric,ratios):
    
    tempdf = pd.DataFrame()
    
    for model in model_list:
        tempdf = tempdf.append(PipelineLayer1(model,metric,ratios))
        
    return tempdf.copy()


In [11]:
# as the functions are currently defined, they only provide a shallow evaluation of which algorithms perform
# best on certain metrics. the best parameters are not output...


# !!
# perhaps we want to return two metrics at a time, so we can see how much we sacrifice in a metric's performance
# in exchange for an optimal value in another metric. as defined now, FinalPipeline returns results for just one metric

In [12]:
LR_GNB = PipelineLayer2([('LR',LogisticRegression()),('GNB',GaussianNB())],metric='neg',ratios=ratios)

In [13]:
LR_GNB

,ADASYN_prec,ADASYN_recall,SMOTE_prec,SMOTE_recall,UnderSamp_prec,UnderSamp_recall,neg_prec,neg_recall
model,,,,,,,,
LR,0.256227,0.715422,0.274333,0.633218,0.273617,0.635457,0.500000,0.000224
reLR,0.256057,0.716094,0.274093,0.631314,0.273093,0.638033,0.500000,0.000224
GNB,0.216318,0.734013,0.252288,0.669840,0.265030,0.611715,0.325714,0.307985
reGNB,0.206968,0.788442,0.235967,0.752828,0.240546,0.743756,0.263459,0.608915


In [14]:
# we see below that for logistic regression, hyperparameter optimization does little to improve the model performance
# the majority of change in the model's performance comes from resampling

# PipelineLayer1(('LR',LogisticRegression()),'neg',ratios)

In [15]:
# for Gaussian NB, regularization has a more notable effect on the model's performance for both the following two metrics

# PipelineLayer1(('GNB',GaussianNB()),'neg',ratios)

In [16]:
# negative precision seems to be the metric most difficult to optimize for.. we can "artifically" improve neg recall
# by increasing the model's sensitivity to negative outputs (via resampling), but it's quite difficult for 
# the model to be precise in its prediction of negative classes